In [7]:
import math
import random

In [8]:
#distance formula 
def l2_distance(x,y):
    return(math.sqrt(sum((xi-yi)**2 for xi,yi in zip(x,y))))

In [12]:
#HNSW graph structure 
class HNSW:
    def __init__(self, M=5, efConstruction=10, ef=10):
        self.M = M
        self.efConstruction = efConstruction
        self.ef = ef
        self.levels = []
        self.vectors = []


#Insert a new vector
    def add(self,vec):
        self.vectors.append(vec)
        idx = len(self.vectors)-1
        # first point creates level 0 graph
        if not self.levels:
            self.levels.append({idx: []})
            return

        # random level assignment (like HNSW)
        L = self._random_level()

        # ensure levels list is long enough
        while len(self.levels) <= L:
            self.levels.append({})

        # search entry point
        entry = 0

        # step 1: greedy descent through higher layers
        for level in reversed(range(L, len(self.levels))):
            entry = self._greedy_search_layer(idx, vec, entry, level)

        # step 2: connect neighbors at each needed level
        for level in range(L + 1):
            self._connect_new_node(idx, vec, entry, level)

    
    # Greedy search in a layer (select closest neighbor)
    
    def _greedy_search_layer(self, idx, vec, entry, level):
        changed = True
        best = entry
        best_dist = l2_distance(vec, self.vectors[entry])

        while changed:
            changed = False
            for nb in self.levels[level].get(best, []):
                d = l2_distance(vec, self.vectors[nb])
                if d < best_dist:
                    best = nb
                    best_dist = d
                    changed = True

        return best

    
    # Connect new node to nearest neighbors
    
    def _connect_new_node(self, idx, vec, entry, level):
        # candidate list for efConstruction
        candidates = self._search_layer(vec, entry, self.efConstruction, level)

        # keep M closest
        neighbors = sorted(candidates, key=lambda j: l2_distance(vec, self.vectors[j]))[:self.M]

        if idx not in self.levels[level]:
            self.levels[level][idx] = []

        for nb in neighbors:
            self.levels[level][idx].append(nb)

            # add reverse edge
            if nb not in self.levels[level]:
                self.levels[level][nb] = []
            self.levels[level][nb].append(idx)

            # trim edges to M
            if len(self.levels[level][nb]) > self.M:
                self.levels[level][nb] = sorted(
                    self.levels[level][nb],
                    key=lambda j: l2_distance(self.vectors[nb], self.vectors[j])
                )[:self.M]

    
    # Search layer using ef parameter
    
    def _search_layer(self, query, entry, ef, level):
        visited = set()
        candidates = [entry]
        best_list = [entry]

        while candidates:
            cur = candidates.pop()
            visited.add(cur)

            for nb in self.levels[level].get(cur, []):
                if nb in visited:
                    continue
                visited.add(nb)

                best_list.append(nb)
                candidates.append(nb)

                if len(best_list) > ef:
                    best_list.sort(key=lambda j: l2_distance(query, self.vectors[j]))
                    best_list = best_list[:ef]

        return best_list

    # random level assignment
    def _random_level(self):
        level = 0
        while random.random() < 0.5:
            level += 1
        return level
    # Final search API (similar to hnswlib)

    def search(self, query, k=5):
        entry = 0

        # descent through top layers (greedy)
        for level in reversed(range(1, len(self.levels))):
            entry = self._greedy_search_layer(None, query, entry, level)

        # bottom layer search (full ef search)
        candidates = self._search_layer(query, entry, self.ef, 0)
        candidates = sorted(candidates, key=lambda j: l2_distance(query, self.vectors[j]))

        return candidates[:k]





In [14]:

# TEST THE HNSW IMPLEMENTATION
import random
random.seed(0)

# create HNSW object
h = HNSW(M=5, efConstruction=10, ef=10)

# insert some vectors
for _ in range(10):
    vec = [random.random(), random.random()]
    h.add(vec)

# search for nearest neighbors
query = [0.5, 0.5]
result = h.search(query, k=3)

print("Nearest neighbors:", result)


Nearest neighbors: [7, 1, 4]
